<a href="https://colab.research.google.com/github/AWH-GlobalPotential-X/AWH-Geo/blob/main/AWH_Geo_threshold_GLDAS_2008_LandScan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Welcome to AWH-Geo Threshold Processor

This tool requires a Google Drive and Earth Engine Account

Click "Connect" at the top right of this notebook.

Then run each of the code blocks below, following instructions.

In [ ]:
#@title Basic setup and earthengine access. Enter your Earth Engine username:

print('Welcome to AWH-Geo Threshold Processor')

# import, authenticate, then initialize EarthEngine module ee
# https://developers.google.com/earth-engine/python_install#package-import
import ee 
print('Make sure the EE version is v0.1.215 or greater...')
print('Current EE version = v' + ee.__version__)
print('')
ee.Authenticate()
ee.Initialize()

worldGeo = ee.Geometry.Polygon( # Created for some masking and geo calcs
  coords=[[-180,-90],[-180,0],[-180,90],[-30,90],[90,90],[180,90],
          [180,0],[180,-90],[30,-90],[-90,-90],[-180,-90]],
  geodesic=False,
  proj='EPSG:4326'
)



In [ ]:
#@title Test Earth Engine connection (see Mt Everest elev and a green map)
# Print the elevation of Mount Everest.
dem = ee.Image('USGS/SRTMGL1_003')
xy = ee.Geometry.Point([86.9250, 27.9881])
elev = dem.sample(xy, 30).first().get('elevation').getInfo()
print('Mount Everest elevation (m):', elev)

# Access H2E assets
from IPython.display import Image
jmpGeofabric_image = ee.Image('projects/h2e/publication/jmpGeofabric_image') # access to H2E folder in EE
Image(url=jmpGeofabric_image.getThumbUrl({'min': 0, 'max': 1, 'dimensions': 512,
                'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']}))



In [ ]:
#@title STEP 1: Export timeseries for given threshold

ee_username = ee.String(ee.Dictionary(ee.List(ee.data.getAssetRoots()).get(0)).get('id'))
ee_username = ee_username.getInfo()

def timeseriesExport(ghi_threshold, rh_threshold):
  
  """
  This script runs threshold values over GLDAS climate variables worldwide, 
  every 3-hours during the five-year period 2008 to 2013. It then converts the 
  resulting imagecollection into a single image with several bands, each of 
  which representing three hourly interval. Finally, it exports this image over 
  yearly tranches and saves each as an EE Image Assets with appropriate names 
  corresponding to the tranche's time period. 
  """
  
  # print the output table code from user input for confirmation
  threshold_pair_name = 'ghi' + ghi_threshold + '_' + 'rh' + rh_threshold
  print('thresholds:', threshold_pair_name)

  # CLIMATE DATA PRE-PROCESSING
  # GLDAS climate dataset used for worldwide (derived) climate metrics
  # https://developers.google.com/earth-engine/datasets/catalog/NASA_GLDAS_V021_NOAH_G025_T3H
  # gldas images in EE catalog
  gldas = ee.ImageCollection('NASA/GLDAS/V021/NOAH/G025/T3H') 
  # print('gldas',gldas.limit(50)) # print some data for inspection (debug)
  gldas_proj = gldas.first().projection() # get GLDAS projection & scale for export
  gldas_scale = gldas_proj.nominalScale()
  print('gldas_scale (should be ~27829):',gldas_scale.getInfo())
  gldas_filtered = gldas.filterDate( # GLDAS climate data
    '2007-12-31','2013-01-01').select( # filter by date
        # filter by GLDAS image collection bands                              
        ['Psurf_f_inst', 'Qair_f_inst', 'SWdown_f_tavg', 'Tair_f_inst']) 
  # print('gldas_filtered',gldas_filtered.limit(50))

  def processTimeseries(i): # core processing algorithm with lookups to outputTable

    i = ee.Image(i) # cast as image
    ghi = i.select('SWdown_f_tavg'
                  ).rename('ghi') # solar global horizontal irradiance [W/m^2]
    rh = ee.Image().expression( # relative humidity calculation [%]
    # from https://earthscience.stackexchange.com/questions/5076/how-to-calculate-specific-humidity-with-relative-humidity-temperature-and-pres?noredirect=1&lq=1
      '0.263 * p * q * (e ** ((17.67 * (T - To)) / (T - 29.65))) ** (-1)', {
        'p': i.select('Psurf_f_inst'), # air pressure [Pa]
        'q': i.select('Qair_f_inst'), # specific humidity or mass mixing ratio of 
                                      # water vapor to total air [dimensionless]
        'e': 2.718281828459045, # Euler's constant
        'T': i.select('Tair_f_inst'), # temperature [K]
        'To': 273.16 # reference temperature [K]
      }).rename('rh')
    
    output = ghi.gte(int(ghi_threshold)).And(rh.gte(int(rh_threshold))) # output in opH/day

    return ee.Image(output.rename('O').setMulti({
        'system:time_start': i.get('system:time_start') # set time as property
      })).updateMask(1) # close partial masks at continental edges

  def outputHourly_export(timeStart, timeEnd, year):

    """
    Run the lookup processing function (from above) across the entire climate 
    timeseries at the finest temporal interval (3 hr for GLDAS). Convert the 
    resulting image collection as a single image with a band for each timestep 
    to allow for export as an Earth Engine asset (you cannot export/save image
    collections as assets).
    """

    # filter GLDAS climate data by time
    gldas_filtered_section = gldas_filtered.filterDate(timeStart, timeEnd)
    # run lookup processor through image timeseries
    outputHourly = gldas_filtered_section.map(processTimeseries) 
    outputHourly_toBands_pre = outputHourly.select(['O']).toBands()
    outputHourly_toBands = outputHourly_toBands_pre.select(
        # input climate variables as multiband image with each band representing timestep
        outputHourly_toBands_pre.bandNames(), 
        outputHourly.toList(1000000).map(
          lambda i: ee.String('H').cat( # "H" for 3-hourly
            ee.String(ee.Date(ee.Image(i).get('system:time_start')).format('yyyyMMddHH')).cat('_O'))
          )
    )

    # notify user of export
    print('Exporting outputHourly year:', year)
    task = ee.batch.Export.image.toAsset(
      image=ee.Image(outputHourly_toBands),
      region=worldGeo,
      description='O_3hourly_' + threshold_pair_name + '_' + year,
      assetId=ee_username + '/O_3hourly_' + threshold_pair_name + '_' + year,
      scale=gldas_scale.getInfo(),
      maxPixels=1e10,
      maxWorkers=2000
    )
    task.start()

  # run timeseries export on entire 3-hr GLDAS for each yearly tranche
  outputHourly_export('2008-01-01','2008-12-31','2008')
  outputHourly_export('2009-01-01','2009-12-31','2009')
  outputHourly_export('2010-01-01','2010-12-31','2010')
  outputHourly_export('2011-01-01','2011-12-31','2011')
  outputHourly_export('2012-01-01','2012-12-31','2012')

thresholds = [
              
              ['400','10'],
              ['400','30'],
              ['400','50'],                            
              ['400','70'],              

              ['500','10'],
              ['500','30'],
              ['500','50'],                           
              ['500','70'],

              ['600','10'],
              ['600','30'],
              ['600','50'],                            
              ['600','70'],

              ['700','10'],
              ['700','30'],
              ['700','50'],                            
              ['700','70'],

]

for threshold_pair in thresholds:
  timeseriesExport(threshold_pair[0],threshold_pair[1])

print('Complete! Read instructions below')

# *Before moving on to the next step... Wait until above tasks are complete in the task manager: https://code.earthengine.google.com/*
(right pane, tab "tasks", click "refresh"; the should show up once the script prints "Exporting...")



In [ ]:
#@title Re-instate earthengine access (follow instructions)

print('Welcome Back to AWH-Geo')
print('')

# import, authenticate, then initialize EarthEngine module ee
# https://developers.google.com/earth-engine/python_install#package-import
import ee 
print('Make sure the EE version is v0.1.215 or greater...')
print('Current EE version = v' + ee.__version__)
print('')
ee.Authenticate()
ee.Initialize()

worldGeo = ee.Geometry.Polygon( # Created for some masking and geo calcs
  coords=[[-180,-90],[-180,0],[-180,90],[-30,90],[90,90],[180,90],
          [180,0],[180,-90],[30,-90],[-90,-90],[-180,-90]],
  geodesic=False,
  proj='EPSG:4326'
)



In [ ]:
#@title STEP 2: Export statistical results for given threshold pair

ee_username = ee.String(ee.Dictionary(ee.List(ee.data.getAssetRoots()).get(0)).get('id'))
ee_username = ee_username.getInfo()

def generateStats(ghi_threshold, rh_threshold):
  
  """
  This function generates single images which contain time-aggregated output statistics. 
  """
  
  # print the output table code from user input for confirmation
  threshold_pair_name = 'ghi' + ghi_threshold + '_' + 'rh' + rh_threshold
  print('thresholds:', threshold_pair_name)

  # CLIMATE DATA PRE-PROCESSING
  gldas = ee.ImageCollection('NASA/GLDAS/V021/NOAH/G025/T3H') # gldas images in EE catalog
  # print('gldas',gldas.limit(50)) # print some data for inspection
  gldas_proj = gldas.first().projection() # get GLDAS projection & scale for export
  gldas_scale = gldas_proj.nominalScale()
  
  # setup the image collection timeseries to chart
  O_2008 = ee.Image(ee_username + '/O_3hourly_' + threshold_pair_name + '_2008') # [L/hr]
  O_2009 = ee.Image(ee_username + '/O_3hourly_' + threshold_pair_name + '_2009')
  O_2010 = ee.Image(ee_username + '/O_3hourly_' + threshold_pair_name + '_2010')
  O_2011 = ee.Image(ee_username + '/O_3hourly_' + threshold_pair_name + '_2011')
  O_2012 = ee.Image(ee_username + '/O_3hourly_' + threshold_pair_name + '_2012')
  # print('O_2010',O_2010);
  
  def unravel(i): # function to "unravel" image bands into an image collection
    def setDate(bandName): # loop over band names in image
      dateCode = ee.Date.parse(
          format='yyyyMMddHH',
          date=ee.String(ee.String(bandName).split('_').get(0)).slice(1) # get date periods from band name
      )
      return i.select([bandName]).rename('O').set('system:time_start',dateCode)
    i = ee.Image(i)
    return i.bandNames().map(setDate)
  # print('testRavel',unravel(O_2012))
  
  # unravel and concatenate all the image stages into a single image collection
  outputTimeseries = ee.ImageCollection(ee.List(
    unravel(O_2008)).cat(
    unravel(O_2009)).cat(
    unravel(O_2010)).cat(
    unravel(O_2011)).cat(
    unravel(O_2012)))
  # print('outputTimeseries',outputTimeseries.limit(50))

  Od_overallMean = outputTimeseries.mean().multiply(24).rename('Od') # hourly output x 24 = mean daily output [L/day]
  
  # export overall daily mean
  task = ee.batch.Export.image.toAsset(
    image=Od_overallMean,
    region=worldGeo,
    description='Od_overallMean_' + threshold_pair_name,
    assetId=ee_username + '/Od_overallMean_GLDAS2008_' + threshold_pair_name,
    scale=gldas_scale.getInfo(),
    maxPixels=1e10,
    maxWorkers=2000
  )
  task.start()
  print('Exporting Od_overallMean_' + threshold_pair_name)

print('Complete! Go to next step.') 

thresholds = [
              ['400','10'],
              ['400','30'],
              ['400','50'],
              ['400','70'],

              ['500','10'],
              ['500','30'],
              ['500','50'],
              ['500','70'],

              ['600','10'],
              ['600','30'],
              ['600','50'],
              ['600','70'],

              ['700','10'],
              ['700','30'],
              ['700','50'],
              ['700','70'],
]

for threshold_pair in thresholds:
  generateStats(threshold_pair[0],threshold_pair[1])


# *Before moving on to the next step... Wait until above tasks are complete in the task manager: https://code.earthengine.google.com/*
(right pane, tab "tasks", click "refresh"; the should show up once the script prints "Exporting...")



In [ ]:
#@title Re-instate earthengine access (follow instructions)

print('Welcome Back to AWH-Geo')
print('')

# import, authenticate, then initialize EarthEngine module ee
# https://developers.google.com/earth-engine/python_install#package-import
import ee 
print('Make sure the EE version is v0.1.215 or greater...')
print('Current EE version = v' + ee.__version__)
print('')
ee.Authenticate()
ee.Initialize()

worldGeo = ee.Geometry.Polygon( # Created for some masking and geo calcs
  coords=[[-180,-90],[-180,0],[-180,90],[-30,90],[90,90],[180,90],
          [180,0],[180,-90],[30,-90],[-90,-90],[-180,-90]],
  geodesic=False,
  proj='EPSG:4326'
)



The process below will sum population results and save as .csv named "results_fc_cumul_gldas_2008_LandScan" in your root Google Drive folder 

In [ ]:
#@title STEP 3: Population without SMDW by threshold pair

ee_username = ee.String(ee.Dictionary(ee.List(ee.data.getAssetRoots()).get(0)).get('id'))
ee_username = ee_username.getInfo()

"""
This script calculates the population without SMDW grouped by the operational 
hours per day of each threshold pairs above. It gathers a set of images which 
have been run through a 5-year hourly climate timeseries as binary on/off device
depending on climate inputs (GHI & rH) and averaged. These images serve as a 
grouping for a reducer over the previously-calculated population without SMDW 
image. The final result is then rearranged as a list of binned sums to be used 
as a histogram of population over the climate variables.
"""

opHour_scale = ee.Image(ee_username + '/Od_overallMean_GLDAS2008_ghi400_rh30'
  ).projection().nominalScale()
print('opHour_scale',opHour_scale.getInfo())

# population water stressed (previously calculated from UN statistics & WorlPop)
pop_wtrSt = ee.Image('projects/h2e/publication/pop_wtrSt_landScan')
pop_wtrSt_scale = pop_wtrSt.projection().nominalScale()
print('pop_wtrSt_scale',pop_wtrSt_scale.getInfo())

# opHours calc
# list the "threshold device" output images into an image collection, round operational hours as integers,
# then collect into an image with several bands (this reduces the calculation time)

opHours_toBands = ee.ImageCollection([
  
  ee.Image(ee_username + '/Od_overallMean_GLDAS2008_ghi400_rh10').rename('opHr_400_rh10').int(),
  ee.Image(ee_username + '/Od_overallMean_GLDAS2008_ghi400_rh30').rename('opHr_400_rh30').int(),
  ee.Image(ee_username + '/Od_overallMean_GLDAS2008_ghi400_rh50').rename('opHr_400_rh50').int(),
  ee.Image(ee_username + '/Od_overallMean_GLDAS2008_ghi400_rh70').rename('opHr_400_rh70').int(),

  ee.Image(ee_username + '/Od_overallMean_GLDAS2008_ghi500_rh10').rename('opHr_500_rh10').int(),
  ee.Image(ee_username + '/Od_overallMean_GLDAS2008_ghi500_rh30').rename('opHr_500_rh30').int(),
  ee.Image(ee_username + '/Od_overallMean_GLDAS2008_ghi500_rh50').rename('opHr_500_rh50').int(),
  ee.Image(ee_username + '/Od_overallMean_GLDAS2008_ghi500_rh70').rename('opHr_500_rh70').int(),  
  
  ee.Image(ee_username + '/Od_overallMean_GLDAS2008_ghi600_rh10').rename('opHr_600_rh10').int(),
  ee.Image(ee_username + '/Od_overallMean_GLDAS2008_ghi600_rh30').rename('opHr_600_rh30').int(),
  ee.Image(ee_username + '/Od_overallMean_GLDAS2008_ghi600_rh50').rename('opHr_600_rh50').int(),
  ee.Image(ee_username + '/Od_overallMean_GLDAS2008_ghi600_rh70').rename('opHr_600_rh70').int(),

  ee.Image(ee_username + '/Od_overallMean_GLDAS2008_ghi700_rh10').rename('opHr_700_rh10').int(),
  ee.Image(ee_username + '/Od_overallMean_GLDAS2008_ghi700_rh30').rename('opHr_700_rh30').int(),
  ee.Image(ee_username + '/Od_overallMean_GLDAS2008_ghi700_rh50').rename('opHr_700_rh50').int(),
  ee.Image(ee_username + '/Od_overallMean_GLDAS2008_ghi700_rh70').rename('opHr_700_rh70').int(),  
  
  ]).toBands()

opHours = opHours_toBands.rename(opHours_toBands.bandNames().map(
    lambda l: ee.String(l).split('_').slice(1).join('_')
  ))
# print('opHours',opHours);

bandNumber = 16 # count the number of threshold images and report it here

# reselect the bands in a new order with water stressed population interlaced -- this allows the reducer to run
selectNumbers = ee.List.repeat(0,bandNumber).zip(ee.List.sequence(1,bandNumber,1)).flatten();
selectNames = ee.List.sequence(1,bandNumber,1).map(
    lambda l: ee.String('pop').cat(ee.Number(l).format('%d'))
  ).zip(opHours.bandNames()).flatten()
# print('selectNumbers',selectNumbers)
# print('selectNames',selectNames)

# run the calculation over the population across each threshold band (with a grouped reducer)
results = pop_wtrSt.addBands(opHours).select( # rearrange bands as noted
  selectNumbers,
  selectNames
).reduceRegion( # run the reducer across population, grouped by operational hours per day of each band / device
  reducer=ee.Reducer.sum().unweighted().group(1,'hours').forEachBand(opHours),
  geometry=worldGeo, 
  scale=pop_wtrSt_scale,
  maxPixels=1e12
)
# print('results',results)

# rearrange results from dictionary to table (i.e. FeatureCollection)
def rearrange_dictTable(key):
  def rearrange_dictTable_1(l):
    return ee.Feature(None, {
      'mktSize': ee.Number(ee.Dictionary(l).get('sum')),
      'rh_threshold': ee.Number.parse(ee.String(ee.String(key).split('_').get(-1)).slice(2)),
      'ghi_threshold': ee.Number.parse(ee.String(key).split('_').get(1)),
      'opHours': ee.Number(ee.Dictionary(l).get('hours'))
    })
  return ee.List(results.get(key)).map(rearrange_dictTable_1)
results_fc = ee.FeatureCollection(results.keys().map(rearrange_dictTable).flatten())
# print('results_fc',results_fc)

# sum the populations within same bin to get cumulative population across operational hours per day
def cumulate(f):
  rh_threshold = ee.Number(ee.Feature(f).get('rh_threshold'))
  ghi_threshold = ee.Number(ee.Feature(f).get('ghi_threshold'))
  opHours = ee.Number(ee.Feature(f).get('opHours'))
  sum = results_fc.filter(ee.Filter.And(
    ee.Filter.eq('rh_threshold', rh_threshold),
    ee.Filter.eq('ghi_threshold', ghi_threshold),
    ee.Filter.gte('opHours', opHours)
    )).aggregate_sum('mktSize')
  return ee.Feature(f).set('mktSize_cumul',sum)
results_fc_cumul = results_fc.map(cumulate)
# print('results_fc_cumul',results_fc_cumul)

# Export to drive
task = ee.batch.Export.table.toDrive(
  collection=results_fc_cumul,
  # folder='', # write Drive folder here if desired
  description='results_fc_cumul_gldas_2008_LandScan',
  fileNamePrefix='results_fc_cumul_gldas_2008_LandScan',
  fileFormat='CSV'
)
task.start()
print('Exporting results_fc_cumul_gldas_2008_LandScan')
print('Complete')

[Go here to see maps](https://code.earthengine.google.com/d8d0036f80832c13355de74eba3a42e4)